In [1]:
import helper
from helper import find_k_nearest_neighbors
from helper import waterkant_festival_description
from helper import Document
import json

In [2]:
time_template="""Extract time-related information such as days of the week, start times, and end times from a question, and format them as specified, follow these steps:

Identify Time Information: Begin by scrutinizing the question to identify mentions of specific days, times, or durations. Key elements to look for include:
Days of the week (e.g., Monday, Tuesday, etc.).
Time expressions (e.g., "9 AM", "1400 hours", "from 9 to 11").
Keywords suggesting time durations (e.g., "morning", "evening", "all day").
List Days of the Week: If any days are mentioned:
Note each day explicitly mentioned. Include all days in a mentioned range (e.g., "Monday to Friday").
Ensure days are fully spelled out (e.g., "Monday", not "Mon") and capitalized.
Determine Start and End Times:
Look for specific times mentioned for starting or ending, noted in either 12-hour or 24-hour format.
Decide if each time refers to a start time or end time based on contextual keywords like "from", "start at", "until", "end by".
Convert times in 12-hour format (AM/PM) into 24-hour format for consistency (e.g., "2 PM" becomes "14:00").
Calculate Minimum Start Time and Maximum End Time:
From all start times identified, choose the earliest as "min_start_time".
From all end times, select the latest as "max_end_time".
Times should be in the "HH:MM" format (e.g., "09:00", "18:00").
Check for Default Values:
If no days are mentioned in the question, default to ['Thursday', 'Friday'].
If no start or end times are provided, use '00:00' as the default "min_start_time" and '23:00' as the "max_end_time".
Format the Output:
Organize the data into a dictionary format with keys: 'days', 'min_start_time', and 'max_end_time'.
Under 'days', include the listed or default days in square brackets.
Assign the respective or default values to 'min_start_time' and 'max_end_time'.
Example:
If the question is "When are late night discussions held?", and no specific days or times are mentioned, your output should look like this:

{
  "days": ["Thursday", "Friday"],
  "min_start_time": "00:00",
  "max_end_time": "23:00"
}
Example:
If the question is "What are the events on Friday starting at 9 AM and ending before 4 PM?", your formatted output should look like this:
{
  "days": ["Friday"],
  "min_start_time": "00:00",
  "max_end_time": "16:00"
}

Here is the question:
%question%
"""

In [3]:
events_template="Here is a list of events which could be helpful to answer the question:\n\n%events%Answer the question based on the information above. Answer in the language of the question. Answer with which events are really fitting to the question and which might not be directly but could also be interesting. Here is the question again:\n%question%"

In [4]:
def replace_placeholders(template, replacements):
    for key, value in replacements.items():
        placeholder = "%" + key + "%"
        template = template.replace(placeholder, value)
    return template

In [5]:
import numpy as np

np_embeddings = np.load('prep/embeddings.npy')

In [6]:
np_embeddings.shape

(112, 1536)

In [7]:
import json
import uuid
with open('prep/documents.json', 'r') as file:
            json_documents = json.load(file)

loaded_documents = [Document.from_json(json_str) for json_str in json_documents]

In [8]:
loaded_documents[0].metadata

{'speakers': [{'name': 'Fabian Böck',
   'role': 'Founder',
   'organization': 'Europe Think Factory'},
  {'name': 'Erdem Ovacik',
   'role': 'Founder and Executive Board Member',
   'organization': 'Donkey Republic'},
  {'name': 'Elina Åkerlind',
   'role': 'CEO and founder',
   'organization': 'Nordic Node AB'},
  {'name': 'Sakina Turabali',
   'role': 'Co-founder/Chief Growth Officer',
   'organization': 'CodeEasy'}],
 'moderators': [{'name': 'Linda-Maraike Plath',
   'role': 'Projektkoordination SEEd Schule (Social Entrepreneurship Education)',
   'organization': 'opencampus.sh'}],
 'type': 'Startup, Panel',
 'date': {'day': 'Friday', 'start_time': '11:00', 'end_time': '11:45'}}

In [9]:
from openai import OpenAI
client = OpenAI()

def get_embedding(text, model="text-embedding-3-small"):
   text = text.replace("\n", " ")
   return client.embeddings.create(input = [text], model=model).data[0].embedding



In [10]:
question="Can you recommend any workshops on artificial intelligence or digital technology on Friday"

In [11]:
question_embedding = get_embedding(question)

In [12]:
np_question_embedding = np.array(question_embedding)

In [13]:
k=len(np_embeddings)
indices, similarities = find_k_nearest_neighbors(np_embeddings, np_question_embedding, k)

In [28]:
from openai import OpenAI
client = OpenAI()
GPT_MODEL = "gpt-3.5-turbo"
def chat_completion_request(messages, tools=None, model=GPT_MODEL, stream=False):
    try:
        response = client.chat.completions.create(
            model=model,
            messages=messages,
            tools=tools,
            temperature = 0.0,
            stream=stream
        )
        return response
    except Exception as e:
        print("Unable to generate ChatCompletion response")
        print(f"Exception: {e}")
        return e



In [29]:
tools = [{
    "type": "function",
    "function": {
        "name": "filter_events_by_day_and_time_constraints",
        "description": "Filters a list of events based on specific day and time constraints, including earliest and latest permissible times for event start and end.",
        "parameters": {
            "type": "object",
            "properties": {
                "target_day": {
                    "type": "string",
                    "description": "The specific day of the week to filter events for, e.g., 'Monday', 'Tuesday'."
                },
                "earliest_start_time": {
                    "type": "string",
                    "description": "The earliest permissible start time for events, in 24-hour format, e.g., '08:00' for 8 AM."
                },
                "latest_end_time": {
                    "type": "string",
                    "description": "The latest permissible ending time for events, if specified, in 24-hour format, e.g., '17:00' for 5 PM."
                },
                "cutoff_start_time": {
                    "type": "string",
                    "description": "The cutoff start time for considering events, any event starting after this time will be excluded, e.g., '23:00'."
                }
            }
        }
    }
}]


In [30]:
tools = [{
    "type": "function",
    "function": {
        "name": "filter_events_by_day_and_time_constraints",
        "description": "Filters a list of events based on specific day and time constraints, including earliest and latest permissible times for event start and end.",
        "parameters": {
            "type": "object",
            "properties": {
                "days": {
                    "type": "string",
                    "description": "required format: python list of the specific days of the week to filter events for, e.g., ['Monday', 'Friday'] or ['Tuesday']. If none are given use ['Thursday', 'Friday'] as the default return value."
                },
                "min_start_time": {
                    "type": "string",
                    "description": "The earliest permissible start time for events, in 24-hour format, e.g., '08:00' for 8 AM. If none is given use '00:00' as the default return value."
                },
                "max_end_time": {
                    "type": "string",
                    "description": "The latest permissible ending time for events, in 24-hour format, e.g., '17:00' for 5 PM. If noen is given use '23:00' as the default return value."
                },
            }
        }
    }
}]


In [31]:
from datetime import datetime

def filter_events(events, filters):
    filtered_events = []
    # Unpacking the filter criteria
    days = filters['days']
    min_start_time_str = filters['min_start_time']
    max_end_time_str = filters['max_end_time']
    
    # Safely parse the global min and max times
    try:
        min_start_time = datetime.strptime(min_start_time_str, "%H:%M").time()
        max_end_time = datetime.strptime(max_end_time_str, "%H:%M").time()
    except ValueError:
        print("Global time filters are not in the correct format (HH:MM). No events will be processed.")
        return filtered_events

    for event in events:
        event_day = event.metadata['date']['day']
        # Safely parse event start and end times
        try:
            event_start_time = datetime.strptime(event.metadata['date']['start_time'], "%H:%M").time()
            event_end_time = datetime.strptime(event.metadata['date']['end_time'], "%H:%M").time()
        except ValueError:
            #print(f"Skipping event due to incorrect time format: {event}")
            continue

        # Check if the event meets all the criteria
        if event_day in days and event_start_time >= min_start_time and event_end_time <= max_end_time:
            filtered_events.append(event)
    
    return filtered_events

In [44]:
filter = {'days': ['Thursday'], 'min_start_time': '00:00', 'max_end_time': '23:00'}

In [45]:
filtered_events = filter_events(loaded_documents, filter)

In [46]:
filtered_events

In [48]:
picked_events = []
counter = 0
for indice in indices:
    if loaded_documents[indice] in filtered_events:
        picked_events.append(loaded_documents[indice])
        counter+=1
        if counter>= 6:
            break

In [49]:
picked_events

In [42]:
messages = []
messages.append({"role": "system", "content": waterkant_festival_description+"You are a helpful chatbot for the attendees of the Waterkant Festival! You answer questions about events, schedules, venue information, transportation options, or anything else. Make no speculations what topics or events are on the festival but call the function then instaed. If you are asked about any topic, event or something that is not directly be answered in a meaningful way by the description above, call the filter_events_by_day_and_time_constraints function. You respond in the same language the user asks the question!"})
messages.append({"role": "user", "content": question})
chat_response = chat_completion_request(messages, tools, stream=True)
#assistant_message = chat_response.choices[0].message

AttributeError: 'Stream' object has no attribute 'choices'

In [43]:
for chunk in chat_response:
    print(chunk.choices[0].tools_call)

ChatCompletionChunk(id='chatcmpl-9NKl8YR3EY837e8uSt3RMeBvsq9KJ', choices=[Choice(delta=ChoiceDelta(content=None, function_call=None, role='assistant', tool_calls=[ChoiceDeltaToolCall(index=0, id='call_HAeJe0w9XFCoppchbqkqvaQW', function=ChoiceDeltaToolCallFunction(arguments='', name='filter_events_by_day_and_time_constraints'), type='function')]), finish_reason=None, index=0, logprobs=None)], created=1715348370, model='gpt-3.5-turbo-0125', object='chat.completion.chunk', system_fingerprint=None)
ChatCompletionChunk(id='chatcmpl-9NKl8YR3EY837e8uSt3RMeBvsq9KJ', choices=[Choice(delta=ChoiceDelta(content=None, function_call=None, role=None, tool_calls=[ChoiceDeltaToolCall(index=0, id=None, function=ChoiceDeltaToolCallFunction(arguments='{"', name=None), type=None)]), finish_reason=None, index=0, logprobs=None)], created=1715348370, model='gpt-3.5-turbo-0125', object='chat.completion.chunk', system_fingerprint=None)
ChatCompletionChunk(id='chatcmpl-9NKl8YR3EY837e8uSt3RMeBvsq9KJ', choices=[C

In [ ]:
messages

In [24]:
assistant_message

ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_1pV08Sb2CudQIxbQhM4ohDRC', function=Function(arguments='{"days":"Friday"}', name='filter_events_by_day_and_time_constraints'), type='function')])

In [25]:
messages = []
messages.append({"role": "system", "content": waterkant_festival_description+"You are a helpful chatbot for the attendees of the Waterkant Festival! You answer questions about events, schedules, venue information, transportation options, or anything else. Make no speculations what topics or events are on the festival but call the function then instaed. If you are asked about any topic, event or something that is not directly be answered in a meaningful way by the description above, call the filter_events_by_day_and_time_constraints function. You respond in the same language the user asks the question!"})
messages.append({"role": "user", "content": question})
chat_response = chat_completion_request(messages, tools)
assistant_message = chat_response.choices[0].message
if assistant_message.tool_calls:
    assistant_message.content = str(assistant_message.tool_calls[0].function)
    messages.append({"role": assistant_message.role, "content": assistant_message.content})
    replacements= {"question": question}
    prompt= replace_placeholders(time_template, replacements)
    time_messages= [{"role": "user", "content": prompt}]
    response = chat_completion_request(time_messages, tools=None, model=GPT_MODEL)
    filters = json.loads(str(response.choices[0].message.content))
    filtered_events = filter_events(loaded_documents, filters)
    picked_events = []
    counter = 0
    for indice in indices:
        if loaded_documents[indice] in filtered_events:
            picked_events.append(loaded_documents[indice])
            counter+=1
            if counter>= 6:
                break
    events_str = ""
    for event in picked_events:
        events_str+= event.content +"\n\n"
    replacements= {"question": question, "events": events_str}
    prompt= replace_placeholders(events_template, replacements)
    events_messages= [{"role": "user", "content": prompt}]
    response = chat_completion_request(events_messages, tools=None, model=GPT_MODEL)
    messages.append({"role": "function", "tool_call_id": assistant_message.tool_calls[0].id, "name": assistant_message.tool_calls[0].function.name, "content": response.choices[0].message.content})

else:
    messages.append({"role": assistant_message.role, "content": assistant_message.content})

In [26]:
assistant_message

ChatCompletionMessage(content='Function(arguments=\'{"days":"Friday"}\', name=\'filter_events_by_day_and_time_constraints\')', role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_2qIZ6RuFJBbHylA1EmjBfYFv', function=Function(arguments='{"days":"Friday"}', name='filter_events_by_day_and_time_constraints'), type='function')])

In [27]:
messages

[{'role': 'system',
  'content': "The Waterkant Festival is a dynamic and innovative event that takes place annually in Kiel, Germany. Since its inception in 2016, the festival has served as a platform for startups, companies, and creative thinkers to showcase and discuss their ideas on technology, sustainability, and future societal structures. The festival is organized by opencampus.sh, a non-profit association aimed at fostering education and entrepreneurship in Schleswig-Holstein.\n\nTypically held over two days, Waterkant Festival features a diverse array of activities including talks, workshops, and interactive exhibitions. Each year, it gathers hundreds of participants and speakers from various fields—ranging from business and science to politics and the arts—to engage in future-oriented discussions and collaborations. The festival's format includes themed sessions that cover a wide range of topics such as artificial intelligence, sustainability, new work methods, and innovative

In [98]:

completion = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=messages,
    temperature=0.0
)

print(completion.choices[0].message.content)

{
  "days": ["Friday"],
  "min_start_time": "00:00",
  "max_end_time": "16:00"
}

Based on your interest in Public Health, the following event might be of particular interest to you:
- **Title:** To fear or not to fear - Visuelle Wissenschaftskommunikation der Antibiotikakrise
  **Description:** Our project aims to bridge the gap between health science and the general public through effective visual science communication. Using the antibiotic resistance crisis as a case study, we have created a visual product that explains the solutions using evolutionary-informed therapies. However, our primary focus is on conducting research on the user experience of the product to gain insights into how to create effective science communication products.
  **Date:** Friday June 16, 2023 10:00 - 10:45 CEST
  **Location:** Ocean Stage

Additionally, you might find the following event interesting:
- **Title:** Ich bin ein Climate Quitter
  **Description:** A very personal story of how I quit my dream j

In [99]:
messages

[{'role': 'system',
  'content': "The Waterkant Festival is a dynamic and innovative event that takes place annually in Kiel, Germany. Since its inception in 2016, the festival has served as a platform for startups, companies, and creative thinkers to showcase and discuss their ideas on technology, sustainability, and future societal structures. The festival is organized by opencampus.sh, a non-profit association aimed at fostering education and entrepreneurship in Schleswig-Holstein.\n\nTypically held over two days, Waterkant Festival features a diverse array of activities including talks, workshops, and interactive exhibitions. Each year, it gathers hundreds of participants and speakers from various fields—ranging from business and science to politics and the arts—to engage in future-oriented discussions and collaborations. The festival's format includes themed sessions that cover a wide range of topics such as artificial intelligence, sustainability, new work methods, and innovative